<a href="https://colab.research.google.com/github/baalasangar/MLAI/blob/master/08_RecommendationSystem/Assignment/recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install surprise


#### 1) Importing Liberaries & reading all data





In [ ]:
import pandas as pd 
pd.set_option('display.max_columns', None)

import numpy as np
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
#%matplotlib inline
sns.set(style="whitegrid")

from sklearn.preprocessing import MinMaxScaler


from surprise import Dataset
from surprise import SVD
from surprise.model_selection.search import GridSearchCV 
from surprise.reader import Reader
from surprise.model_selection import train_test_split
from surprise.model_selection.split import KFold 
import surprise.accuracy as accuracy
from surprise.model_selection.validation import cross_validate
import random
from surprise import KNNWithMeans


from collections import defaultdict




In [ ]:
basepath =  "/RecommendationSys/"
#basepath = "/content/drive/MyDrive/Colab Notebooks/Greatelearning/RecommendationSys/"

data_rev1 = pd.read_csv(basepath+"phone_user_review_file_1.csv",encoding='latin-1')
data_rev2 = pd.read_csv(basepath+"phone_user_review_file_2.csv",encoding='latin-1')
data_rev3 = pd.read_csv(basepath+"phone_user_review_file_3.csv",encoding='latin-1')
data_rev4 = pd.read_csv(basepath+"phone_user_review_file_4.csv",encoding='latin-1')
data_rev5 = pd.read_csv(basepath+"phone_user_review_file_5.csv",encoding='latin-1')
data_rev6 = pd.read_csv(basepath+"phone_user_review_file_6.csv",encoding='latin-1')




In [ ]:
# Check the Details for the data
data_list = [data_rev1,data_rev2,data_rev3,data_rev4,data_rev5,data_rev6]
for data_object in data_list:
  print(data_object.columns)
  print(data_object.head())

##### Merge the provided CSVs into one data-frame.


In [ ]:
data = pd.concat(data_list)
data.reset_index(inplace=True)
data.drop(columns=["index"],inplace=True)
data.tail(n=10000)


#### Exploratory Data Analysis

In [ ]:
# Create a copy of the data for analysis
data_analysis = data.copy()

##### Check a few observations and shape of the data-frame.


In [ ]:
print("Shape of the data - ",data_analysis.shape)
print("Columns of data - ",data_analysis.columns)

In [ ]:
# Random check on data for 5o records
data_analysis.sample(n=25,random_state=612)

##### formating the phone url Columns

In [ ]:
# formating the phone url Columns

# Check is all the rows starts with "cellphones"
print(data_analysis['phone_url'].str.startswith('/cellphones/').value_counts() )
data_analysis['phone_url'] = data_analysis['phone_url'].str.replace('/cellphones/','')
data_analysis['phone_url'] = data_analysis['phone_url'].str[:-1]
data_analysis['phone_url'].value_counts()

##### Analysis the Column "author" 

In [ ]:
#pd.set_option('display.max_rows', None)
print("Not of observation with empty - ",data_analysis["author"].isna().sum())
print("Not of observation with empty % - ",(data_analysis["author"].isna().sum() / data_analysis.shape[0])*100) 

Since only 1000000 are request for analysis, removing the observation where author names are empty





 

In [ ]:

data_analysis.dropna(subset=["author"],inplace=True)
data_analysis.isna().sum()

##### Analysis the Column "Score and score_max"

In [ ]:
##### Analysis the Column "Score and score_max"

In [ ]:
print("Not of observation with empty - ",data_analysis["score"].isna().sum())
print("Not of observation with empty % - ",(data_analysis["score"].isna().sum() / data_analysis.shape[0])*100) 

Since only 1000000 are request for analysis, removing the observation where score names are empty

In [ ]:
data_analysis.dropna(subset=["score"],inplace=True)
data_analysis.isna().sum()

In [ ]:
# is there any observation with score greater than max score ?
data_analysis.query("score > score_max").count().sum()


 - is there any observation with score greater than max score ? -NO

###### Round off scores to the nearest integers



In [ ]:
data_analysis["score"] = np.round(data_analysis["score"])
data_analysis["score"].unique()

In [ ]:
# is the scoring is done on same scale  ( 1 -10) ?  
data_analysis["score_max"].value_counts()

- is the scoring is done on same scale  ( 1 -10) ?  - YES

##### Dropping irrelevant features & removing duplicate columns

In [ ]:
# Removing the duplicate rows - when phone_url,author & score are same
rows_with_duplicate = int(data_analysis.shape[0])

print("# of Observations - ",rows_with_duplicate)
data_analysis.drop_duplicates(subset=['phone_url','author','score'],keep='first',inplace=True)

rows_without_duplicate = int(data_analysis.shape[0])
print("# of Observations after removing duplicate - ",rows_without_duplicate)

print("% of rows duplicated rows removed - " , ((rows_with_duplicate - rows_without_duplicate)/rows_with_duplicate)*100)

- 13% duplicated rows have been removed , it's OK since only 10L rows are to be considered for the analysis. 

In [ ]:
# 'phone_url','author','score' are considered for the analysis on other rows are removed.
data_analysis = data_analysis[['phone_url','author','score']]
data_analysis.head()

##### Sampling 1000000 rows

In [ ]:
data_analysis = data_analysis.sample(n=1000000,random_state=612)
print("Check the shape ", data_analysis.shape)
print(" missing values ", data_analysis.isna().sum().sum())

### 2) Answer the following questions 

1.   Identify the most rated features
2.   Identify the users with most number of reviews.
3.   Select the data with products having more than 50 ratings and users who have given more than 50 ratings. Report the shape of the final
dataset

##### most rated features

In [ ]:
data_groupby_phone = data_analysis.groupby(by="phone_url").count()
data_groupby_phone = data_groupby_phone.sort_values(by=['author'],ascending=False)
data_groupby_phone["author"].head(10)

 ##### users with most number of reviews

In [ ]:
data_groupby_author = data_analysis.groupby(by="author").count()
data_groupby_author = data_groupby_author.sort_values(by='phone_url',ascending=False)
data_groupby_author['phone_url'].head(10)

#### Filter Data - product with 50 rating and User gave 50 rating

In [ ]:
data_groupby_phone = data_groupby_phone.query("author > 50")
data_groupby_author = data_groupby_author.query("phone_url > 50")
data_analysis = data_analysis[ data_analysis["phone_url"].isin(data_groupby_phone.index) | data_analysis["author"].isin(data_groupby_author.index)]
print("Data Size selected for Model Building-- ",data_analysis.shape)
print("Columns Selected -- ",data_analysis.columns)
data_analysis.head()

#### User & Product

In [ ]:
print("Unique products - ", data_analysis['phone_url'].nunique())
print("No of USers  - ", data_analysis['author'].nunique())

### **popularity based model**

In [ ]:
rating_mean_count =  pd.DataFrame(data_analysis.groupby(by="phone_url")["score"].mean())
rating_mean_count["rating_count"] =  pd.DataFrame(data_analysis.groupby(by="phone_url")["score"].count())

##### Caculating Weighted Score 

W = ((R*V) + (C*M)) / ( V+M )
 - R = Average score for a product
 - V = total number of counts recived for product
 - M = min votes required to be listed ( 90 % quantile )
 - C = Average vote across the whole product

In [ ]:
R = rating_mean_count["score"]
V = rating_mean_count["rating_count"]
M = rating_mean_count["rating_count"].quantile(0.9)
C = rating_mean_count["score"].mean() #Product to be considered only when it has more rating count than 90% of the product
rating_mean_count["weigted_score"]  = ( (R*V)+ (C*M) / ( V+M ))

In [ ]:
# minmaxscale = MinMaxScaler()
# rating_mean_count["score_nor"] = minmaxscale.fit_transform(rating_mean_count[["score"]])
# rating_mean_count["rating_count_nor"] = minmaxscale.fit_transform(rating_mean_count[["rating_count"]])
# rating_mean_count["weigted_score_nor"] = minmaxscale.fit_transform(rating_mean_count[["weigted_score"]])

In [ ]:
plt.figure(figsize=(12,6))
vis_data = rating_mean_count.sort_values(by=["weigted_score","rating_count","score"],ascending=False)
vis_data.reset_index(inplace=True)
sns.barplot(data=vis_data.head(10) ,y="phone_url",x="weigted_score",orient="h")
plt.xlabel("Weighted Score")
plt.ylabel("Phone id")
plt.show()

In [ ]:
def popularity_recommendation(fileter_col_list,top_n):
  top_n_item = []
  return rating_mean_count.sort_values(by=fileter_col_list,ascending=False).index[:top_n]

##### Top 5 recommendation

In [ ]:
popularity_recommendation(["weigted_score","rating_count","score"],5)

### **collaborative filtering**

In [ ]:
def get_top_n(predictions, n=10):
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, details in predictions:
        if details['was_impossible'] == False:
            top_n[uid].append((iid, est))
        
        

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n


##### Impute outliers

In [ ]:
print(data_analysis["score"].value_counts())
data_analysis["score"].value_counts(normalize=True)*100

 - Rating count for (0,1,3) are less than 1% which can be assigned with 2 rating.
 - Rating count of ( 5,6) are less than 1% which can be assigned to 6

In [ ]:
data_analysis["score_imp"]  = np.where(data_analysis["score"].isin([0,1,3]),2, data_analysis["score"])
data_analysis["score_imp"] = np.where(data_analysis["score_imp"].isin([5,7]),6,data_analysis["score_imp"])
data_analysis["score_imp"].value_counts(normalize=True)*100

##### User-user collabrative filtering

###### Train Model with Cross Validation - Without treating outlier (Score)

In [ ]:
# reducing the data size to 10K considering limitation computation resources
data_analysis_CF =  data_analysis[['author','phone_url','score']].sample(random_state=612,n=5000)

reader = Reader(rating_scale=(0, 5))
user_base_data = Dataset.load_from_df(data_analysis_CF,reader)
param_grid = {'k': [10,15,20,25,30,35,40,45,50],
              'bsl_options': {'method': ['als', 'sgd'],
                              'reg': [1, 2]},
              'sim_options': {'name': ['pearson_baseline', 'cosine'],
                              'user_based': [True]}              }
kmean_gs_user_base = GridSearchCV(KNNWithMeans, param_grid, measures=['rmse'], cv=KFold(n_splits=3,random_state=123),return_train_measures = True) 
kmean_gs_user_base.fit(user_base_data)


Model performance

In [ ]:
print("Best Score" , kmean_gs_user_base.best_score['rmse'])
print("best hyperparameter", kmean_gs_user_base.best_params['rmse'])
results_df = pd.DataFrame.from_dict(kmean_gs_user_base.cv_results)
results_df[["mean_train_rmse","std_train_rmse","mean_test_rmse","std_test_time"]].sort_values(by=["mean_test_rmse","mean_train_rmse"])


###### Train Model with Cross Validation - With treating outlier (Score)

In [ ]:
# reducing the data size to 10K considering limitation computation resources
data_analysis_CF =  data_analysis[['author','phone_url','score_imp']].sample(random_state=612,n=5000)

reader = Reader(rating_scale=(0, 5))
user_base_data = Dataset.load_from_df(data_analysis_CF,reader)
param_grid = {'k': [10,15,20,25,30,35,40,45,50],
              'bsl_options': {'method': ['als', 'sgd'],
                              'reg': [1, 2]},
              'sim_options': {'name': ['pearson_baseline', 'cosine'],
                              'user_based': [True]}              }
kmean_gs_user_base_v1 = GridSearchCV(KNNWithMeans, param_grid, measures=['rmse'], cv=KFold(n_splits=3,random_state=123),return_train_measures = True) 
kmean_gs_user_base_v1.fit(user_base_data)

In [ ]:
print("Best Score" , kmean_gs_user_base_v1.best_score['rmse'])
print("best hyperparameter", kmean_gs_user_base_v1.best_params['rmse'])
results_df = pd.DataFrame.from_dict(kmean_gs_user_base_v1.cv_results)
results_df[["mean_train_rmse","std_train_rmse","mean_test_rmse","std_test_time"]].sort_values(by=["mean_test_rmse","mean_train_rmse"])

Observation :
No Major difference on the model performance when capping the score value

##### predicting with best Model & Top 5 recommendation

In [ ]:
data_analysis_CF =  data_analysis[['author','phone_url','score']].sample(random_state=612,n=100)
reader = Reader(rating_scale=(0, 5))
user_base_data = Dataset.load_from_df(data_analysis_CF,reader)
trainset = user_base_data.build_full_trainset()
testset = trainset.build_anti_testset(fill=np.nan)
best_userbase = kmean_gs_user_base.best_estimator['rmse']
best_userbase.fit(trainset)
userbase_prediction = best_userbase.test(testset)

In [ ]:
userbase_model_pred = pd.DataFrame([[i.uid, i.iid, i.est,i.details['was_impossible']] for i in userbase_prediction], columns=['user_id', 'movie_id', 'score',"was_impossible"])
userbase_model_pred.head()

In [ ]:
test_users = userbase_model_pred.sample(n=10,random_state=123)
def user_base_recommendation(user_list,n=5):
  recommendation = dict();
  for user_id in user_list:
    movie_list = userbase_model_pred[userbase_model_pred.user_id == user_id].sort_values(by=["score"],ascending=False).head(n=n)["movie_id"]
    recommendation[user_id] = list(movie_list)
  return recommendation

user_base_recommendation(test_users["user_id"])

In [ ]:
results_df = pd.DataFrame.from_dict(gs.cv_results)

print("Best Score" , kmean_gs_user_base.best_score['rmse'])
print("best hyperparameter", kmean_gs_user_base.best_params['rmse'])
best_kmean = KNNWithMeans(k=kmean_gs.best_params['rmse']['k'],sim_options = kmean_gs.best_params['rmse']['sim_options'])
best_kmean.fit(trainset)
test_pred = algo.test(testset)
print("Test Set - RMSE")
accuracy.rmse(test_pred, verbose=True)

top_n = get_top_n(test_pred, n=5) 
test_user = ["Amazon Customer","Cliente Amazon","Anonymous","Amazon Kunde","Client d'Amazon","David","Alex","Daniel","Chris","Marco"]
for test_customer in test_user:
 print(test_customer," - ", [iid for (iid, _) in top_n[test_customer]])  

###### Using the Score Imputation

In [ ]:
reader = Reader(rating_scale=(0, 5),line_format='user item rating')

user_user_sample_data = data_analysis[['author','phone_url','score_imp']].sample(random_state=612,n=10000)


user_user_data = Dataset.load_from_df(user_user_sample_data,reader)
trainset, testset = train_test_split(user_user_data, test_size=.3,random_state=123)

param_grid = {'k': [10,15,20,25,30,35,40,45,50],
              'sim_options': {'name': ['pearson_baseline', 'cosine'],
                              'user_based': [True]}              }
kmean_gs = GridSearchCV(KNNWithMeans, param_grid, measures=['rmse'], cv=5) 
kmean_gs.fit(user_user_data)

print("Best Score" , kmean_gs.best_score['rmse'])
print("best hyperparameter", kmean_gs.best_params['rmse'])
# best_kmean = KNNWithMeans(k=kmean_gs.best_params['rmse']['k'],sim_options = kmean_gs.best_params['rmse']['sim_options'])
# best_kmean.fit(trainset)
# test_pred = algo.test(testset)
# print("Test Set - RMSE")
# accuracy.rmse(test_pred, verbose=True)

In [ ]:
best_kmean = KNNWithMeans(k=kmean_gs.best_params['rmse']['k'],sim_options = kmean_gs.best_params['rmse']['sim_options'])
best_kmean.fit(trainset)
test_pred = algo.test(testset)
print("Test Set - RMSE")
accuracy.rmse(test_pred, verbose=True)

In [ ]:
top_n = get_top_n(test_pred, n=5) 
test_user = ["Amazon Customer","Cliente Amazon","Anonymous","Amazon Kunde","Client d'Amazon","David","Alex","Daniel","Chris","Marco"]
for test_customer in test_user:
 print(test_customer," - ", [iid for (iid, _) in top_n[test_customer]])  

#### Item Based Collaboration model

In [ ]:
reader = Reader(rating_scale=(0, 5),line_format='user item rating')

user_user_sample_data = data_analysis[['author','phone_url','score']].sample(random_state=612,n=10000)


user_user_data = Dataset.load_from_df(user_user_sample_data,reader)
trainset, testset = train_test_split(user_user_data, test_size=.3,random_state=123)

param_grid = {'k': [10,15,20,25,30,35,40,45,50],
              'sim_options': {'name': ['pearson_baseline', 'cosine'],
                              'user_based': [False]}              }
kmean_gs = GridSearchCV(KNNWithMeans, param_grid, measures=['rmse'], cv=5) 
kmean_gs.fit(user_user_data)

In [ ]:
print("Best Score" , kmean_gs.best_score['rmse'])
print("best hyperparameter", kmean_gs.best_params['rmse'])
best_kmean = KNNWithMeans(k=kmean_gs.best_params['rmse']['k'],sim_options = kmean_gs.best_params['rmse']['sim_options'])
best_kmean.fit(trainset)
test_pred = algo.test(testset)
print("Test Set - RMSE")
accuracy.rmse(test_pred, verbose=True)

top_n = get_top_n(test_pred, n=5) 
test_user = ["Amazon Customer","Cliente Amazon","Anonymous","Amazon Kunde","Client d'Amazon","David","Alex","Daniel","Chris","Marco"]
for test_customer in test_user:
 print(test_customer," - ", [iid for (iid, _) in top_n[test_customer]]) 

##### Using the Score Imputation

In [ ]:
reader = Reader(rating_scale=(0, 5),line_format='user item rating')

user_user_sample_data = data_analysis[['author','phone_url','score_imp']].sample(random_state=612,n=10000)


user_user_data = Dataset.load_from_df(user_user_sample_data,reader)
trainset, testset = train_test_split(user_user_data, test_size=.3,random_state=123)

param_grid = {'k': [10,15,20,25,30,35,40,45,50],
              'sim_options': {'name': ['pearson_baseline', 'cosine'],
                              'user_based': [False]}              }
kmean_gs = GridSearchCV(KNNWithMeans, param_grid, measures=['rmse'], cv=5) 
kmean_gs.fit(user_user_data)

In [ ]:
print("Best Score" , kmean_gs.best_score['rmse'])
print("best hyperparameter", kmean_gs.best_params['rmse'])

best_kmean = KNNWithMeans(k=kmean_gs.best_params['rmse']['k'],sim_options = kmean_gs.best_params['rmse']['sim_options'])
best_kmean.fit(trainset)
test_pred = algo.test(testset)
print("Test Set - RMSE")
accuracy.rmse(test_pred, verbose=True)

top_n = get_top_n(test_pred, n=5) 
test_user = ["Amazon Customer","Cliente Amazon","Anonymous","Amazon Kunde","Client d'Amazon","David","Alex","Daniel","Chris","Marco"]
for test_customer in test_user:
 print(test_customer," - ", [iid for (iid, _) in top_n[test_customer]])  

#### Matrix Factorization ( using SVD )

In [ ]:
# reader = Reader(rating_scale=(0, 5),line_format='user item rating')
# matrix_fact_data = Dataset.load_from_df(data_analysis[['author','phone_url','score']],reader)
# raw_ratings = matrix_fact_data.raw_ratings
# random.shuffle(raw_ratings)

# # 90% trainset, 10% testset                                                
# threshold = int(.9 * len(raw_ratings))                                     
# trainset_raw_ratings = raw_ratings[:threshold]                             
# test_raw_ratings = raw_ratings[threshold:]
# matrix_fact_data.raw_ratings = trainset_raw_ratings


# param_grid = {'n_factors':[5,10,15,50],'n_epochs':[10,20]}
# svd_gs = GridSearchCV(SVD, param_grid, measures=['rmse'],cv=KFold(n_splits=5, random_state=123),n_jobs=5)
# svd_gs.fit(matrix_fact_data)


In [ ]:
# print("Score " , svd_gs.best_score)
# print("Best Param " , svd_gs.best_params)

In [ ]:
# svd_bestmodel = SVD(n_factors =svd_gs.best_params['rmse']['n_factors'],n_epochs = svd_gs.best_params['rmse']['n_epochs'])
# trainset = matrix_fact_data.build_full_trainset()                                      
# svd_bestmodel.fit(trainset)   


# # now test on the trainset                                                 
# testset = matrix_fact_data.construct_testset(trainset_raw_ratings)                     
# predictions = svd_bestmodel.test(testset)                                           
# print('Accuracy on the trainset:')                                         
# accuracy.rmse(predictions)                                                 
                                                                           
# # now test on the testset                                                  
# testset = matrix_fact_data.construct_testset(test_raw_ratings)                         
# predictions = svd_bestmodel.test(testset)                                           
# print('Accuracy on the testset:')                                          
# accuracy.rmse(predictions)  


###### Train Model with cross validation

In [ ]:
reader = Reader(rating_scale=(0, 5),line_format='user item rating')
matrix_fact_data = Dataset.load_from_df(data_analysis[['author','phone_url','score']],reader)
raw_ratings = matrix_fact_data.raw_ratings
threshold = int(.9 * len(raw_ratings))                                     
trainset_raw_ratings = raw_ratings[:threshold]                             
test_raw_ratings = raw_ratings[threshold:]
matrix_fact_data.raw_ratings = trainset_raw_ratings
svd = SVD()
results = cross_validate(svd,matrix_fact_data,measures=['rmse'],cv=5,return_train_measures =True,n_jobs =-1)
results


###### RMSE - for Train / Validate / Test

In [ ]:
print("Average train RMSE -", results['train_rmse'].mean())
print("Average Validate RMSE -", results['test_rmse'].mean())

In [ ]:
trainset = matrix_fact_data.build_full_trainset()
svd.fit(trainset)
testset = matrix_fact_data.construct_testset(test_raw_ratings)  
predict = svd.test(testset)
predict

In [ ]:
print("Test RMSE - ",accuracy.rmse(predict))

###### Top 5 recommendation for test user

In [ ]:
top_n = get_top_n(predict, n=5) 
test_user = ["Amazon Customer","Cliente Amazon","Anonymous","Amazon Kunde","Client d'Amazon","David","Alex","Daniel","Chris","Marco"]
for test_customer in test_user:
 print(test_customer," - ", [iid for (iid, _) in top_n[test_customer]])  

In [ ]:
reader = Reader(rating_scale=(0, 5),line_format='user item rating')
df = data_analysis[['author','phone_url','score']].sample(random_state=612,n=10)
data1 = Dataset.load_from_df(df,reader)
df

In [ ]:
trainset = data1.build_full_trainset()

In [ ]:
trainset.build_anti_testset()